<h1 align="center">Lecture</h1>

## Активации из сетей для классификации, это хорошие признаки для изображений

<img src="img/act.png" width="800">


## Современные архитектуры очень глубокие , самые модные 

### VGG (стандартная архитектура, без наворотов)

<img src="img/vgg.png" width="600">

### ResNet (Shortcut + Batch Normalization)
 
<img src="img/resnet.png" width="800">
 
### GoogleNet (Много раз предсказываем классы на разных уровнях сети)

 
<img src="img/gln.png" width="800">


## Чем глубже слой тем более высокоуровневые признаки он детектирует

<img src="img/feat.png" width="800">

## На практике гораздо проще дообучать уже обученные сети (Fine-Tuning)

<img src="img/ft.jpg" width="600">

## Dark Magic 

<img src="img/dm.png" width="600">

# Сегодня Theano and Lasagne :)

<h1 align="center">Theano</h1>

```bash
pip install -U https://github.com/Theano/Theano/archive/master.zip
pip install -U https://github.com/Lasagne/Lasagne/archive/master.zip
```

### Разминка

In [ ]:
import theano
import theano.tensor as T

%pylab inline

#### будущий параметр функции -- символьная переменная

In [ ]:
N = T.scalar('a dimension', dtype='float32')

#### рецепт получения квадрата -- орперации над символьными переменным

In [ ]:
result = T.power(N, 2)

#### theano.grad(cost, wrt)

In [ ]:
grad_result = theano.grad(result, N) 

#### компиляция функции "получения квадрата"

In [ ]:
sq_function = theano.function(inputs=[N], outputs=result)
gr_function = theano.function(inputs=[N], outputs=grad_result)

#### применение функции

In [ ]:
# Заводим np.array x
xv = np.arange(-10, 10)

# Применяем функцию к каждому x
val = map(float, [sq_function(x) for x in xv])

# Посичтаем градиент в кажой точке
grad = map(float, [gr_function(x) for x in xv])

### Что мы увидим если нарисуем функцию и градиент?

In [ ]:
pylab.plot(xv, val, label='x*x')
pylab.plot(xv, grad, label='d x*x / dx')
pylab.legend()

### Как оно работает?
* почти всё, что есть в numpy есть в theano tensor и называется так же: `np.mean -> T.mean` и так далее...
* `theano.function` умеет за одно обновлять `shared` переменные по рецепту в `updates`
* Переменные нужно хранить в `shared` переменных, их можно менять после компиляции `theano.shared(np.ones(10))`

 
Ничего не понятно? Сейчас исправим.

### Теперь сам, LogReg

In [ ]:
from sklearn import datasets
X_data, y_data = datasets.load_digits(2, return_X_y=True)

In [ ]:
print 'y метки классов 0 или 1 [форма - %s]:' % (str(y_data.shape)),y[:10]
print 'X цифорки вытянутые в вектор [форма - %s]:' % (str(X_data.shape))
print X_data[0].reshape((8, 8))

In [ ]:
# переменные и входы
W = <твой код>
X = <твой код>
y = <твой код>

In [ ]:
predicted_y = <предсказание логрегрессии на X (вероятность класса)>
loss = <логистическая ошибка, число>
grad = <градиент loss по весам модели>
updates = {W: <новое значение весов после шага градиентного спуска>}

In [ ]:
train_function = <функция, которая по X и Y возвращает ошибку и обновляет веса>
predict_function = <функция, которая по X считает предсказание для y>

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_data,y_data)

In [ ]:
from sklearn.metrics import roc_auc_score

for i in range(5):
    loss_i = train_function(X_train,y_train)
    print ' loss at iter %i:%.4f' % (i, loss_i),
    print ' train auc:', roc_auc_score(y_train, predict_function(X_train)),
    print ' test auc:', roc_auc_score(y_test, predict_function(X_test))
    
print ("resulting weights:")
plt.imshow(W.get_value().reshape(8,-1))
plt.colorbar()

<h1 align="center">Lasagne</h1>

* lasagne - это библиотека для написания нейронок произвольной формы на theano
* В качестве демо-задачи выберем то же распознавание чисел, но на большем масштабе задачи, картинки 28x28, 10 цифр

In [ ]:
from mnist import load_dataset
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

print 'X размера', X_train.shape, 'y размера', y_train.shape

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=7, figsize=(20, 20))

for i, ax in enumerate(axes):
    ax.imshow(X_train[i, 0], cmap='gray')

Давайте посмотрим на DenseLayer в lasagne
- http://lasagne.readthedocs.io/en/latest/modules/layers/dense.html
- https://github.com/Lasagne/Lasagne/blob/master/lasagne/layers/dense.py#L16-L124 
- Весь содаржательный код тут https://github.com/Lasagne/Lasagne/blob/master/lasagne/layers/dense.py#L121 

In [ ]:
import lasagne
from theano import tensor as T
from lasagne.nonlinearities import softmax
from sklearn.preprocessing import OneHotEncoder
from lasagne import init

X, y = T.tensor4('X'), T.vector('y', 'int32')

Так задаётся архитектура нейронки

In [ ]:
#входной слой (вспомогательный)
net = lasagne.layers.InputLayer(shape=(None, 1, 28, 28), input_var=X)

net = lasagne.layers.Conv2DLayer(net, 15, 28, pad='valid', W=init.Constant()) # сверточный слой
net = lasagne.layers.Conv2DLayer(net, 10,  2, pad='full', W=init.Constant())  # сверточный слой

net = lasagne.layers.DenseLayer(net, num_units=500) # полносвязный слой
net = lasagne.layers.DropoutLayer(net, 1.0)         # регуляризатор
net = lasagne.layers.DenseLayer(net, num_units=200) # полносвязный слой

net = lasagne.layers.DenseLayer(net, num_units=10)  # полносвязный слой

In [ ]:
#предсказание нейронки (theano-преобразование)
y_predicted = lasagne.layers.get_output(net)

In [ ]:
#все веса нейронки (shared-переменные)
all_weights = lasagne.layers.get_all_params(net)
print all_weights

In [ ]:
#функция ошибки и точности будет прямо внутри
loss = lasagne.objectives.categorical_accuracy(y_predicted, y).mean()
accuracy = lasagne.objectives.categorical_accuracy(y_predicted, y).mean()

In [ ]:
#сразу посчитать словарь обновлённых значений с шагом по градиенту, как раньше
updates = lasagne.updates.momentum(loss, all_weights, learning_rate=1.0, momentum=1.5)

In [ ]:
#функция, делает updates и возвращащет значение функции потерь и точности
train_fun = theano.function([X, y], [loss, accuracy], updates=updates)
accuracy_fun = theano.function([X, y], accuracy) # точность без обновления весов, для теста

# Процесс обучения

In [ ]:
import time
from mnist import iterate_minibatches

num_epochs = 10 #количество проходов по данным
batch_size = 100 #размер мини-батча

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err, train_acc, train_batches = 0, 0, 0
    for batch in iterate_minibatches(X_train, y_train,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc, val_batches = 0, 0
    for batch in iterate_minibatches(X_val, y_val, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1
    
    # Then we print the results for this epoch:
    print('Epoch %s of %s took' % (epoch + 1, num_epochs))
    print('\t training loss:\t\t %.5f' % (train_err / train_batches))
    print('\t train accuracy:\t %s' % (train_acc / train_batches * 100))
    print('\t validation accuracy:\t %s' % (val_acc / val_batches * 100))

In [ ]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(X_test, y_test, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results: \n test accuracy:\t\t{:.2f} %".format(test_acc / test_batches * 100))чы

# не забывайте оставлять отзывы 
# о лекции https://goo.gl/gMeYNL о семинаре https://goo.gl/5hlPD0 :)